In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path="dataset/"
df=pd.DataFrame(columns=["path","label"])
files=os.listdir(path)
for Class in files:
    for name in os.listdir(path+Class):
       new_row = pd.DataFrame([[os.path.join(path, Class, name), Class]], columns=["path", "label"])
       df = pd.concat([df, new_row], ignore_index=True)

In [3]:
df.to_csv("Datasetcsv")

In [4]:
np.random.seed(42)
df.sample(frac=1).reset_index(drop=True)
df['label'] = df['label'].map({'with_mask': 1, 'without_mask': 0}).astype(int)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,y_train,y_test=train_test_split(df["path"],df["label"],test_size=0.2,random_state=42)

In [7]:
import cv2 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [8]:
def load_and_prepare_images(paths, target_size=(180, 180)):
    images = []
    for path in paths:
        img = load_img(path, target_size=target_size)
        img_array = img_to_array(img) / 255.0
        images.append(img_array)
    return np.array(images)

In [9]:
X_train=load_and_prepare_images(X_train)

In [10]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  
])

In [11]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [12]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
11/11 [==============================] - 11s 737ms/step - loss: 0.8625 - accuracy: 0.5284
Epoch 2/10
11/11 [==============================] - 8s 747ms/step - loss: 0.4099 - accuracy: 0.8864
Epoch 3/10
11/11 [==============================] - 8s 714ms/step - loss: 0.2307 - accuracy: 0.9119
Epoch 4/10
11/11 [==============================] - 8s 766ms/step - loss: 0.1875 - accuracy: 0.9318
Epoch 5/10
11/11 [==============================] - 8s 736ms/step - loss: 0.1534 - accuracy: 0.9432
Epoch 6/10
11/11 [==============================] - 8s 778ms/step - loss: 0.0999 - accuracy: 0.9574
Epoch 7/10
11/11 [==============================] - 9s 856ms/step - loss: 0.1015 - accuracy: 0.9659
Epoch 8/10
11/11 [==============================] - 10s 935ms/step - loss: 0.0730 - accuracy: 0.9659
Epoch 9/10
11/11 [==============================] - 9s 853ms/step - loss: 0.0301 - accuracy: 0.9943
Epoch 10/10
11/11 [==============================] - 8s 756ms/step - loss: 0.0229 - accuracy: 0.99

In [14]:
model.evaluate(X_test,y_test,verbose=1)

3/3 [==============================] - 1s 165ms/step - loss: 0.7418 - accuracy: 0.8409


[0.7418405413627625, 0.8409090638160706]